For this project I create a voronoi diagram on the map based on data points (or point of interests), voronoi diagram have applications in almost all areas of science and engineering. For geospatial use case, it is useful to tell us the closest point of interest (POI) by representing each POI with a dot inside a polygon shape. So within a polygon, the closest POI is definitely the dot inside the polygon.

Ok, let's start the project. As usual, for the first step let's import all necessary packages

In [ ]:
import folium
import numpy as np
import pandas as pd
from shapely.geometry import Point, LineString
import geopandas as gpd
from geopandas import GeoDataFrame

Insert data points & create delaunay triangulation using those points

In [25]:

points = np.array([[-6.305286, 106.644085], [-6.301309, 106.653261], [-6.284774, 106.637751], [-6.284598, 106.665062], [-6.283521, 106.627582],[-6.276593, 106.641365], [-6.303643, 106.625972]])
from scipy.spatial import Delaunay
tri = Delaunay(points)

Create two dataframes, one to save every edges of the triangulation and the other one to save circumcenter/centroid for each triangle

In [48]:
df = pd.DataFrame()
circumcenter = pd.DataFrame()

In [49]:
for x in range(points[tri.simplices].shape[0]):
    df = df.append(pd.DataFrame({'group': x+1,"geometry": [LineString([Point(points[tri.simplices][x][0][1], points[tri.simplices][x][0][0]), Point(points[tri.simplices][x][1][1], points[tri.simplices][x][1][0])])]}))
    df = df.append(pd.DataFrame({'group': x+1,"geometry": [LineString([Point(points[tri.simplices][x][1][1], points[tri.simplices][x][1][0]), Point(points[tri.simplices][x][2][1], points[tri.simplices][x][2][0])])]}))
    df = df.append(pd.DataFrame({'group': x+1,"geometry": [LineString([Point(points[tri.simplices][x][2][1], points[tri.simplices][x][2][0]), Point(points[tri.simplices][x][0][1], points[tri.simplices][x][0][0])])]}))
    # find lat & lon coordinate for three vertices of the triangle
    circumcenter = circumcenter.append(pd.DataFrame({'group': x+1,
                                                     'point_1_lon': points[tri.simplices][x][0][1],
                                                     'point_1_lat': points[tri.simplices][x][0][0],
                                                     'point_2_lon': points[tri.simplices][x][1][1],
                                                     'point_2_lat': points[tri.simplices][x][1][0],
                                                     'point_3_lon': points[tri.simplices][x][2][1],
                                                     'point_3_lat': points[tri.simplices][x][2][0]
                                                    }, index=[0]))


In [30]:
circumcenter

,group,point_1_lon,point_1_lat,point_2_lon,point_2_lat,point_3_lon,point_3_lat
0,1,106.665062,-6.284598,106.637751,-6.284774,106.641365,-6.276593
0,2,106.653261,-6.301309,106.637751,-6.284774,106.665062,-6.284598
0,3,106.637751,-6.284774,106.653261,-6.301309,106.644085,-6.305286
0,4,106.637751,-6.284774,106.627582,-6.283521,106.641365,-6.276593
0,5,106.625972,-6.303643,106.637751,-6.284774,106.644085,-6.305286
0,6,106.627582,-6.283521,106.637751,-6.284774,106.625972,-6.303643


Find mid point for each edge

In [31]:
df.reset_index(drop=True)
df['mid_point'] = df.apply(lambda row: row['geometry'].interpolate(0.5, normalized = True),axis=1)

Convert dataframe into geodataframe

In [32]:
gdf = GeoDataFrame(df,geometry='geometry')

Create to find circumcenter position of each triangle

In [33]:
def circumcenter_triangle(ax,ay,bx,by,cx,cy):
    
    x_ab_point = (ax + bx)/2
    y_ab_point = (ay + by)/2
    slope_ab = (by-ay)/(bx-ax)
    slope_ab_perpendicular = -1 / slope_ab
    y_intercept_ab = y_ab_point - (slope_ab_perpendicular*x_ab_point)
    
    x_bc_point = (bx + cx)/2
    y_bc_point = (by + cy)/2
    slope_bc = (cy-by)/(cx-bx)
    slope_bc_perpendicular = -1 / slope_bc
    y_intercept_bc = y_bc_point - (slope_bc_perpendicular*x_bc_point)
    
    x_ca_point = (cx + ax)/2
    y_ca_point = (cy + ay)/2
    slope_ca = (ay-cy)/(ax-cx)
    slope_ca_perpendicular = -1 / slope_ca
    y_intercept_ca = y_ca_point - (slope_ca_perpendicular*x_ca_point)
    
    
    circumcenter_x = (y_intercept_ca - y_intercept_bc)/(slope_bc_perpendicular - slope_ca_perpendicular)
    circumcenter_y = (slope_ca_perpendicular*circumcenter_x)+y_intercept_ca
    
    
    
#     circumcenter_y = (slope_bc_perpendicular*x_bc_point)+y_intercept_bc
#     circumcenter_x = (circumcenter_y - y_intercept_ab)/slope_ab_perpendicular
#     d = 2 * (ax * (by - cy) + bx * (cy - ay) + cx * (ay - by))
#     ux = ((ax * ax + ay * ay) * (by - cy) + (bx * bx + by * by) * (cy - ay) + (cx * cx + cy * cy) * (ay - by)) / d
#     uy = ((ax * ax + ay * ay) * (cx - bx) + (bx * bx + by * by) * (ax - cx) + (cx * cx + cy * cy) * (bx - ax)) / d
    return Point(circumcenter_x, circumcenter_y)

Since circumcenter is not always inside the triangle (and this is happen in this project). I decided to create another function to find centroid instead.

In [34]:
def centroid_triangle(ax,ay,bx,by,cx,cy):
      
    # Formula to calculate centroid  
    x = (ax + bx + cx) / 3
    y = (ay + by + cy) / 3
    
    return Point(x, y)

Find circumcenter/centroid for each triangle

In [35]:

circumcenter['circumcenter_triangle'] = circumcenter.apply(lambda row: centroid_triangle(row['point_1_lon'],row['point_1_lat'],row['point_2_lon'],row['point_2_lat'],row['point_3_lon'],row['point_3_lat']) ,axis=1)

Find coordinate for each circumcenter

In [36]:

circumcenter['circumcenter_triangle_lon'] = circumcenter.apply(lambda row: row['circumcenter_triangle'].x,axis=1)
circumcenter['circumcenter_triangle_lat'] = circumcenter.apply(lambda row: row['circumcenter_triangle'].y,axis=1)

Get lon lat for each mid points

In [50]:

gdf['mid_point_lon'] = gdf.apply(lambda row: row['mid_point'].x,axis=1)
gdf['mid_point_lat'] = gdf.apply(lambda row: row['mid_point'].y,axis=1)

In [38]:
circumcenter

,group,point_1_lon,point_1_lat,point_2_lon,point_2_lat,point_3_lon,point_3_lat,circumcenter_triangle,circumcenter_triangle_lon,circumcenter_triangle_lat
0,1,106.665062,-6.284598,106.637751,-6.284774,106.641365,-6.276593,POINT (106.6480593333333 -6.281988333333334),106.648059,-6.281988
0,2,106.653261,-6.301309,106.637751,-6.284774,106.665062,-6.284598,POINT (106.6520246666667 -6.290226999999999),106.652025,-6.290227
0,3,106.637751,-6.284774,106.653261,-6.301309,106.644085,-6.305286,POINT (106.6450323333333 -6.297122999999999),106.645032,-6.297123
0,4,106.637751,-6.284774,106.627582,-6.283521,106.641365,-6.276593,POINT (106.635566 -6.281629333333332),106.635566,-6.281629
0,5,106.625972,-6.303643,106.637751,-6.284774,106.644085,-6.305286,POINT (106.635936 -6.297901),106.635936,-6.297901
0,6,106.627582,-6.283521,106.637751,-6.284774,106.625972,-6.303643,POINT (106.630435 -6.290646),106.630435,-6.290646


Create new dataframe to save perpendicular lines

In [39]:

perpendicular_line = pd.merge(gdf[['group','mid_point','mid_point_lon','mid_point_lat']], circumcenter[['group','circumcenter_triangle','circumcenter_triangle_lon','circumcenter_triangle_lat']], on='group')
perpendicular_line['geometry'] = perpendicular_line.apply(lambda row:LineString([row['mid_point'], row['circumcenter_triangle']]),axis=1)
perpendicular_line = GeoDataFrame(perpendicular_line,geometry='geometry')

Find slope & y-intercept

In [40]:

perpendicular_line['slope'] = perpendicular_line.apply(lambda row: (row['mid_point_lat']-row['circumcenter_triangle_lat'])/(row['mid_point_lon']-row['circumcenter_triangle_lon']),axis=1)
perpendicular_line['y_intercept'] = perpendicular_line.apply(lambda row: row['mid_point_lat'] - (row['slope'] * row['mid_point_lon']),axis=1)


Determine the coverage area of the voronoi diagram

In [42]:

area_max_lon = 106.670929
area_min_lon = 106.619602
area_max_lat = -6.275713
area_min_lat = -6.309795

coords = [[area_min_lon, area_min_lat], [ area_min_lon, area_max_lat], [ area_max_lon,area_max_lat], [ area_max_lon,area_min_lat]]


Extend the perpendicular lines so it can fit the size of area

In [43]:


perpendicular_line.loc[perpendicular_line['circumcenter_triangle_lat']>perpendicular_line['mid_point_lat'], 'ext_lat'] = area_min_lat
perpendicular_line.loc[perpendicular_line['circumcenter_triangle_lat']<perpendicular_line['mid_point_lat'], 'ext_lat'] = area_max_lat

perpendicular_line['ext_lon'] = perpendicular_line.apply(lambda row: (row['ext_lat']-row['y_intercept'])/row['slope'],axis=1)

perpendicular_line.loc[(perpendicular_line['ext_lon']>area_max_lon) , 'ext_lon'] = area_max_lon
perpendicular_line.loc[(perpendicular_line['ext_lon']<area_min_lon) , 'ext_lon'] = area_min_lon

perpendicular_line['ext_lat'] = perpendicular_line.apply(lambda row: (row['ext_lon']*row['slope'])+row['y_intercept'],axis=1)


perpendicular_line.loc[perpendicular_line.duplicated(['mid_point_lon', 'mid_point_lat'],keep=False), 'ext_lon'] = perpendicular_line['mid_point_lon']
perpendicular_line.loc[perpendicular_line.duplicated(['mid_point_lon', 'mid_point_lat'],keep=False), 'ext_lat'] = perpendicular_line['mid_point_lat']



In [44]:
perpendicular_line['ext_point'] = perpendicular_line.apply(lambda row:Point(row['ext_lon'],row['ext_lat']),axis=1)


perpendicular_line['geometry'] = perpendicular_line.apply(lambda row:LineString([row['ext_point'], row['circumcenter_triangle']]),axis=1)
perpendicular_line = GeoDataFrame(perpendicular_line,geometry='geometry')


In [45]:
perpendicular_line

,group,mid_point,mid_point_lon,mid_point_lat,circumcenter_triangle,circumcenter_triangle_lon,circumcenter_triangle_lat,geometry,slope,y_intercept,ext_lat,ext_lon,ext_point
0,1,POINT (106.6514065 -6.284686),106.651407,-6.284686,POINT (106.6480593333333 -6.281988333333334),106.648059,-6.281988,"LINESTRING (106.65141 -6.28469, 106.64806 -6.2...",-0.805955,79.671579,-6.284686,106.651407,POINT (106.6514065 -6.284686)
1,1,POINT (106.639558 -6.2806835),106.639558,-6.280684,POINT (106.6480593333333 -6.281988333333334),106.648059,-6.281988,"LINESTRING (106.63956 -6.28068, 106.64806 -6.2...",-0.153486,10.086967,-6.280684,106.639558,POINT (106.639558 -6.2806835)
2,1,POINT (106.6532135 -6.2805955),106.653213,-6.280596,POINT (106.6480593333333 -6.281988333333334),106.648059,-6.281988,"LINESTRING (106.67093 -6.27581, 106.64806 -6.2...",0.270234,-35.101967,-6.275808,106.670929,POINT (106.670929 -6.275808161810822)
3,2,POINT (106.645506 -6.293041499999999),106.645506,-6.293041,POINT (106.6520246666667 -6.290226999999999),106.652025,-6.290227,"LINESTRING (106.64551 -6.29304, 106.65202 -6.2...",0.431760,-52.338313,-6.293041,106.645506,POINT (106.645506 -6.293041499999999)
4,2,POINT (106.6514065 -6.284686),106.651407,-6.284686,POINT (106.6520246666667 -6.290226999999999),106.652025,-6.290227,"LINESTRING (106.65141 -6.28469, 106.65202 -6.2...",-8.963602,949.696080,-6.284686,106.651407,POINT (106.6514065 -6.284686)
5,2,POINT (106.6591615 -6.292953499999999),106.659162,-6.292953,POINT (106.6520246666667 -6.290226999999999),106.652025,-6.290227,"LINESTRING (106.67093 -6.29745, 106.65202 -6.2...",-0.382032,34.454279,-6.297449,106.670929,POINT (106.670929 -6.297449063683686)
6,3,POINT (106.645506 -6.293041499999999),106.645506,-6.293041,POINT (106.6450323333333 -6.297122999999999),106.645032,-6.297123,"LINESTRING (106.64551 -6.29304, 106.64503 -6.2...",8.616819,-925.238079,-6.293041,106.645506,POINT (106.645506 -6.293041499999999)
7,3,POINT (106.648673 -6.303297499999999),106.648673,-6.303297,POINT (106.6450323333333 -6.297122999999999),106.645032,-6.297123,"LINESTRING (106.65250 -6.30980, 106.64503 -6.2...",-1.695981,174.570782,-6.309795,106.652504,POINT (106.6525041169596 -6.309795000000008)
8,3,POINT (106.640918 -6.29503),106.640918,-6.295030,POINT (106.6450323333333 -6.297122999999999),106.645032,-6.297123,"LINESTRING (106.64092 -6.29503, 106.64503 -6.2...",-0.508709,47.954206,-6.295030,106.640918,POINT (106.640918 -6.29503)
9,4,POINT (106.6326665 -6.2841475),106.632666,-6.284147,POINT (106.635566 -6.281629333333332),106.635566,-6.281629,"LINESTRING (106.63267 -6.28415, 106.63557 -6.2...",0.868483,-98.892813,-6.284147,106.632666,POINT (106.6326665 -6.2841475)


In [ ]:
Display the voronoi diagram with folium 

In [46]:
gdf.crs = {'init' :'epsg:4326'}
perpendicular_line.crs = {'init' :'epsg:4326'}
m = folium.Map((-6.304029, 106.645874),
zoom_start=13,
tiles="CartoDb dark_matter")

#draw the area
line = {'type': 'Feature',
        'geometry': {'type': 'LineString',
                     'coordinates': coords }
        }

folium.GeoJson(line, name='line',
               style_function=lambda x: {'color': '#999999',
                                         'fill': '#999999',
                                         'opacity': 0.8,
                                         'fillOpacity': 0.5}).add_to(m)

#draw the voronoi diagram
folium.GeoJson(perpendicular_line['geometry'], style_function=lambda x: {'color': '#e6e31d', 'weight':'1'}).add_to(m)


#draw the data points
locs = points
for location in locs:
    folium.CircleMarker(location=location, 
        color = "#F4F6F7",   radius=0.01).add_to(m)
    


m

/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
